# **Uploading the files and concating the DFs**

In [1]:
import pandas as pd
import numpy as np
import re
import statistics

In [2]:
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','z']

In [3]:
#df_a = pd.read_csv("../BEEV/letter_scraped/"+ 'a' +"_scraped.csv")                #Remember to ask Cristina about yellow stuff after backslash
#df_a

In [4]:
#Building a dicto_scraped with all DF to be able to concat everything
dicto_scraped = {letter : pd.read_csv("../BEEV/letter_scraped/"+ letter +"_scraped.csv",index_col=None) for letter in letters}

C:\Users\Utilizador\anaconda3\envs\Safe space\lib\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (56,80,95,110,111,112,116,121,122,123,127,128,129,130,131,132,133,134,135,136,137,138,139,142) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\Utilizador\anaconda3\envs\Safe space\lib\site-packages\IPython\core\interactiveshell.py:3364: DtypeWarning: Columns (55,56,108,118,119,123,124,125,126,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,149,150,151,152,153,154,155,156,157,158,159,160,167,168,169,170,171) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
#Building a list of shape to check coherence between letter.csv and letter_scarped_csv
scraped_shape = [dicto_scraped[let].shape[0] for let in dicto_scraped]

In [6]:
#Building a dicto_csv with all url list to be able to compare with dicto_scraped
dicto_csv = {letter : pd.read_csv("../BEEV/letter_csvs/"+ letter +".csv",index_col=None) for letter in letters}

In [7]:
#Building a list of shape to check coherence between letter.csv and letter_scarped_csv
csv_shape = [dicto_csv[lett].shape[0] for lett in dicto_csv.keys()]

In [8]:
#Calculating the difference between both lenght list
scraped_shape
for i in range(len(scraped_shape)):
    print(scraped_shape[i] - csv_shape[i])  

-14
-13
-14
-6
0
-21
-5
-15
-3
-4
-7
-7
-35
-11
-3
-7
0
-7
-10
-18
0
-10
-7
0
0


In [9]:
#The difference come from the empty dataframe we scrapped so we have to count the number of empty DF to see if it macth
# test calculation --->  
#dicto_csv['a']['dataframe'][dicto_csv['a']['dataframe'].str.contains('Empty') == True].shape[0]
empty_df_list = [dicto_csv[letter]['dataframe'][dicto_csv[letter]['dataframe'].str.contains('Empty') == True].shape[0] for letter in letters]
    

In [10]:
#Calculating the difference between both lenght list and the number of empty DF
# If every number is equal to 0 we are ok ! next step will be to check the model coherence
for i in range(len(scraped_shape)):
    print(scraped_shape[i] - csv_shape[i] + empty_df_list[i])  

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


# **GEV dataframe**

In [11]:
#Building the DF using the DF in the dicto_scraped
df = pd.concat([dicto_scraped[letter] for letter in letters])

In [12]:
df_gev = df[df['Powertrain Architecture'] == 'Internal Combustion engine'].copy()
df_gev.reset_index(drop=False, inplace=True)
df_gev.drop(columns = ['index'], inplace =True)

In [13]:
#Definition of the key word for filtering columns
words_wanted4_column = ['Brand','Model','Generation', 'Engine', 'production','Powertrain','Body','Fuel','CO2','Power']


In [14]:
#Checking manually to see which key words we need for the columns
#df_gev.columns[79:110]
len(list(df_gev.columns))

192

In [15]:
#Building the list of columns containing the key words
col_filtered = []
for word in words_wanted4_column:
  for w in list(df_gev.columns):
    if w.lower().find(word.lower()) != -1:
      col_filtered.append(w)
len(col_filtered)

94

In [16]:
#filtering fuel columns
fuel_list =['Fuel consumption','fuel consumption']
fuel_col_filtered = []
for word in fuel_list:
  for w in col_filtered:
    if w.find(word) != -1:
      fuel_col_filtered.append(w)
len(fuel_col_filtered)
fuel_col_filtered

['Fuel consumption (economy) - urban',
 'Fuel consumption (economy) - extra urban',
 'Fuel consumption (economy) - combined',
 'Fuel consumption (economy) - urban (NEDC)',
 'Fuel consumption (economy) - extra urban (NEDC)',
 'Fuel consumption (economy) - combined (NEDC)',
 'Fuel consumption at Low speed (WLTP)',
 'Fuel consumption at Medium speed (WLTP)',
 'Fuel consumption at high speed (WLTP)',
 'Fuel consumption at very high speed (WLTP)',
 'Fuel consumption (economy) - urban (LPG)',
 'Fuel consumption (economy) - extra urban (LPG)',
 'Fuel consumption (economy) - combined (LPG)',
 'Fuel consumption (economy) - urban (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - extra urban (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - combined (NEDC, WLTP equivalent)',
 'Fuel consumption at Low speed (WLTP) (CNG)',
 'Fuel consumption at Medium speed (WLTP) (CNG)',
 'Fuel consumption at high speed (WLTP) (CNG)',
 'Fuel consumption at very high speed (WLTP) (CNG)',
 'Fuel consum

## CO2 cleaning

In [17]:
#filtering CO2 columns
co2_list =['CO2 emissions']
co2_col_filtered = []
for word in co2_list:
  for w in col_filtered:
    if w.find(word) != -1:
      co2_col_filtered.append(w)
len(co2_col_filtered)
print(co2_col_filtered)
co2_col_filtered[3].split()

['CO2 emissions', 'CO2 emissions (NEDC)', 'CO2 emissions (WLTP)', 'CO2 emissions (LPG)', 'CO2 emissions (NEDC, WLTP equivalent)', 'CO2 emissions (CNG)', 'CO2 emissions (CNG) (NEDC)', 'CO2 emissions (Ethanol - E85)', 'CO2 emissions (LPG) (NEDC)', 'CO2 emissions (LPG) (NEDC, WLTP equivalent)', 'CO2 emissions (LPG) (WLTP)', 'CO2 emissions (CNG) (WLTP)', 'CO2 emissions (CNG) (NEDC, WLTP equivalent)']


['CO2', 'emissions', '(LPG)']

In [18]:
#Grabbing the interesting part in the column name
for w in co2_col_filtered:
    print(" ".join(w.split()[2:]))

def co2_grab(dfserie):
    return " ".join(dfserie.name.split()[2:])


(NEDC)
(WLTP)
(LPG)
(NEDC, WLTP equivalent)
(CNG)
(CNG) (NEDC)
(Ethanol - E85)
(LPG) (NEDC)
(LPG) (NEDC, WLTP equivalent)
(LPG) (WLTP)
(CNG) (WLTP)
(CNG) (NEDC, WLTP equivalent)


In [19]:
df_gev[['CO2 emissions',
 'CO2 emissions (NEDC)',
 'CO2 emissions (WLTP)',
 'CO2 emissions (LPG)',
 'CO2 emissions (NEDC, WLTP equivalent)',
 'CO2 emissions (CNG)',
 'CO2 emissions (CNG) (NEDC)',
 'CO2 emissions (Ethanol - E85)',
 'CO2 emissions (LPG) (NEDC)',
 'CO2 emissions (LPG) (NEDC, WLTP equivalent)',
 'CO2 emissions (LPG) (WLTP)',
 'CO2 emissions (CNG) (WLTP)',
 'CO2 emissions (CNG) (NEDC, WLTP equivalent)']]#.isna().sum()

,CO2 emissions,CO2 emissions (NEDC),CO2 emissions (WLTP),CO2 emissions (LPG),"CO2 emissions (NEDC, WLTP equivalent)",CO2 emissions (CNG),CO2 emissions (CNG) (NEDC),CO2 emissions (Ethanol - E85),CO2 emissions (LPG) (NEDC),"CO2 emissions (LPG) (NEDC, WLTP equivalent)",CO2 emissions (LPG) (WLTP),CO2 emissions (CNG) (WLTP),"CO2 emissions (CNG) (NEDC, WLTP equivalent)"
0,153 g/km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,148 g/km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,153 g/km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,148 g/km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,151 g/km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_co2 = df_gev[co2_col_filtered].copy()
for column in co2_col_filtered:
    df_co2[column] = df_gev[[column]].apply(lambda item: item + co2_grab(item))
df_co2['co2_clean'] = df_co2.fillna('').sum(axis=1)

In [21]:
#df_co2['co2_clean'][200:250]
df_co2['co2_clean'] = df_co2['co2_clean'].apply(lambda item: np.nan if len(item) ==0 else item)


In [22]:
#df_co2.isna().sum()
#df_co2['co2_clean'].iloc[200].split()[0]
df_co2['co2_nb'] = df_co2['co2_clean'].apply(lambda item: np.nan if isinstance(item, float) else (item.replace('l','1').split()[0] if item.replace('l','1').split()[0].find('-') == -1 else item.split()[0].split('-')[0]))
df_co2['co2_unit'] = df_co2['co2_clean'].apply(lambda item: np.nan if isinstance(item, float) else item.split()[1])
df_co2['co2_nb'] = df_co2['co2_nb'].apply(lambda item: float(item) if (not isinstance(item, float) and item.isdigit()) else item) #.str.isnumeric()
df_co2[['co2_clean','co2_nb','co2_unit']]


,co2_clean,co2_nb,co2_unit
0,153 g/km,153.0,g/km
1,148 g/km,148.0,g/km
2,153 g/km,153.0,g/km
3,148 g/km,148.0,g/km
4,151 g/km(NEDC),151.0,g/km(NEDC)
...,...,...,...
43107,NaN,NaN,NaN
43108,NaN,NaN,NaN
43109,NaN,NaN,NaN
43110,NaN,NaN,NaN


In [23]:
df_gev_co2 = df_gev[['Brand', 'Model', 'Generation', 'Modification (Engine)', 'Body type', 'Fuel tank capacity', 'Fuel consumption (economy) - combined', 'Fuel Type', 'CO2 emissions', 'Power']].merge(df_co2[['co2_clean','co2_nb','co2_unit']],left_index=True,right_index=True)

In [24]:
df_gev_co2.head()

,Brand,Model,Generation,Modification (Engine),Body type,Fuel tank capacity,Fuel consumption (economy) - combined,Fuel Type,CO2 emissions,Power,co2_clean,co2_nb,co2_unit
0,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp) Automatic,Roadster,45 l 11.89 US gal | 9.9 UK gal,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,Petrol (Gasoline),153 g/km,170 Hp @ 5500 rpm.,153 g/km,153.0,g/km
1,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp),Roadster,45 l 11.89 US gal | 9.9 UK gal,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,Petrol (Gasoline),148 g/km,170 Hp @ 5500 rpm.,148 g/km,148.0,g/km
2,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp) Automatic,Cabriolet,45 l 11.89 US gal | 9.9 UK gal,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,Petrol (Gasoline),153 g/km,170 Hp @ 5500 rpm.,153 g/km,153.0,g/km
3,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp),Cabriolet,45 l 11.89 US gal | 9.9 UK gal,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,Petrol (Gasoline),148 g/km,170 Hp @ 5500 rpm.,148 g/km,148.0,g/km
4,Abarth,500,500C,1.4 T-Jet (140 Hp) MTA,Cabriolet,35 l 9.25 US gal | 7.7 UK gal,NaN,Petrol (Gasoline),NaN,140 Hp @ 5500 rpm.,151 g/km(NEDC),151.0,g/km(NEDC)


In [25]:
#Function 2 split co2
def split_bin(value):
    if value.find('-') != -1:
        value = value.split('-')
        value[0] = float(value[0])
        #print(value[1])
        if len(value[1])>1:
            value[1] = float(value[1])
        else: value[1] = float(value[0])
        return value
    elif value.find('+') != -1:
        value = value.split('+')
        value[0] = float(value[0])
        value[1] = float(value[1])
        return value
    else: return value
    
#Function 2 clean fuel
def basic_split_bin(value):
    if value.find('-') != -1:
        value = value.split('-')
        if len(value[1])==0:
            value[1] = value[0]
        return value
    elif value.find('+') != -1:
        value = value.split('+')
        return value
    else: return value

In [26]:
#Cleaning Fuel tank capacity in a new column
df_gev_co2['Fuel_tank_capacity_clean_(liters)'] = df_gev_co2['Fuel tank capacity'].apply(lambda item: item.split()[0] if (not isinstance(item, float)) else item)
#and (item.split()[0].find('-') == -1 or item.split()[0].find('+') == -1 )
df_gev_co2['Fuel_tank_capacity_clean_(liters)'] = df_gev_co2['Fuel_tank_capacity_clean_(liters)'].apply(lambda item: float(item) if (not isinstance(item, float) and item.replace('+','-').find('-') == -1) else item) #and item.isdigit()) else item) #.str.isnumeric()
df_gev_co2['Fuel_tank_capacity_clean_(liters)'] = df_gev_co2['Fuel_tank_capacity_clean_(liters)'].apply(lambda item: split_bin(item) if (not isinstance(item, float)) else item)
df_gev_co2.head()

,Brand,Model,Generation,Modification (Engine),Body type,Fuel tank capacity,Fuel consumption (economy) - combined,Fuel Type,CO2 emissions,Power,co2_clean,co2_nb,co2_unit,Fuel_tank_capacity_clean_(liters)
0,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp) Automatic,Roadster,45 l 11.89 US gal | 9.9 UK gal,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,Petrol (Gasoline),153 g/km,170 Hp @ 5500 rpm.,153 g/km,153.0,g/km,45.0
1,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp),Roadster,45 l 11.89 US gal | 9.9 UK gal,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,Petrol (Gasoline),148 g/km,170 Hp @ 5500 rpm.,148 g/km,148.0,g/km,45.0
2,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp) Automatic,Cabriolet,45 l 11.89 US gal | 9.9 UK gal,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,Petrol (Gasoline),153 g/km,170 Hp @ 5500 rpm.,153 g/km,153.0,g/km,45.0
3,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp),Cabriolet,45 l 11.89 US gal | 9.9 UK gal,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,Petrol (Gasoline),148 g/km,170 Hp @ 5500 rpm.,148 g/km,148.0,g/km,45.0
4,Abarth,500,500C,1.4 T-Jet (140 Hp) MTA,Cabriolet,35 l 9.25 US gal | 7.7 UK gal,NaN,Petrol (Gasoline),NaN,140 Hp @ 5500 rpm.,151 g/km(NEDC),151.0,g/km(NEDC),35.0


## Fuel cleaning

### Creating the dataframes

In [27]:
df_fuel = df_gev[['Fuel consumption (economy) - urban', 'Fuel consumption (economy) - extra urban','Fuel consumption (economy) - combined',
 'Fuel consumption (economy) - urban (NEDC)','Fuel consumption (economy) - extra urban (NEDC)','Fuel consumption (economy) - combined (NEDC)',
 'Fuel consumption (economy) - urban (LPG)','Fuel consumption (economy) - extra urban (LPG)','Fuel consumption (economy) - combined (LPG)',
 'Fuel consumption (economy) - urban (NEDC, WLTP equivalent)','Fuel consumption (economy) - extra urban (NEDC, WLTP equivalent)','Fuel consumption (economy) - combined (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - urban (CNG) (NEDC)','Fuel consumption (economy) - extra urban (CNG) (NEDC)','Fuel consumption (economy) - combined (CNG) (NEDC)',
 'Fuel consumption (economy) - urban (CNG)','Fuel consumption (economy) - extra urban (CNG)','Fuel consumption (economy) - combined (CNG)',
 'Fuel consumption (economy) - urban (Ethanol - E85)','Fuel consumption (economy) - extra urban (Ethanol - E85)','Fuel consumption (economy) - combined (Ethanol - E85)',
 'Fuel consumption (economy) - urban (LPG) (NEDC)','Fuel consumption (economy) - extra urban (LPG) (NEDC)', 'Fuel consumption (economy) - combined (LPG) (NEDC)',
 'Fuel consumption (economy) - urban (LPG) (NEDC, WLTP equivalent)','Fuel consumption (economy) - extra urban (LPG) (NEDC, WLTP equivalent)','Fuel consumption (economy) - combined (LPG) (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - urban (CNG) (NEDC, WLTP equivalent)','Fuel consumption (economy) - extra urban (CNG) (NEDC, WLTP equivalent)','Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)']]
#-------------------------------------------------------------------------
fuel_combined = ['combined']
col_combined = []
for word in fuel_combined:
  for w in list(df_fuel.columns):
    if w.lower().find(word.lower()) != -1:
      col_combined.append(w)
#print(col_combined)

df_fuel_combined = df_gev[['Fuel consumption (economy) - combined',
 'Fuel consumption (economy) - combined (NEDC)',
 'Fuel consumption (economy) - combined (LPG)',
 'Fuel consumption (economy) - combined (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - combined (CNG) (NEDC)',
 'Fuel consumption (economy) - combined (CNG)',
 'Fuel consumption (economy) - combined (Ethanol - E85)',
 'Fuel consumption (economy) - combined (LPG) (NEDC)',
 'Fuel consumption (economy) - combined (LPG) (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)']]
#print(df_fuel_combined.head())
#-------------------------------------------------------------------------
fuel_urban = ['urban']
col_urban = []
for word in fuel_urban:
  for w in list(df_fuel.columns):
    if w.lower().find(word.lower()) != -1 and "extra" not in w:
      col_urban.append(w)
#print(col_urban)

df_fuel_urban = df_gev[['Fuel consumption (economy) - urban',
 'Fuel consumption (economy) - urban (NEDC)',
 'Fuel consumption (economy) - urban (LPG)',
 'Fuel consumption (economy) - urban (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - urban (CNG) (NEDC)',
 'Fuel consumption (economy) - urban (CNG)',
 'Fuel consumption (economy) - urban (Ethanol - E85)',
 'Fuel consumption (economy) - urban (LPG) (NEDC)',
 'Fuel consumption (economy) - urban (LPG) (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - urban (CNG) (NEDC, WLTP equivalent)']]
#print(df_fuel_urban)
#----------------------------------------------------------------------------
fuel_extra_urban = ['extra urban']
col_fuel_extra_urban = []
for word in fuel_extra_urban:
  for w in list(df_fuel.columns):
    if w.lower().find(word.lower()) != -1:
      col_fuel_extra_urban.append(w)
#print(col_fuel_extra_urban)

df_fuel_extra_urban = df_gev[['Fuel consumption (economy) - extra urban',
 'Fuel consumption (economy) - extra urban (NEDC)',
 'Fuel consumption (economy) - extra urban (LPG)',
 'Fuel consumption (economy) - extra urban (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - extra urban (CNG) (NEDC)',
 'Fuel consumption (economy) - extra urban (CNG)',
 'Fuel consumption (economy) - extra urban (Ethanol - E85)',
 'Fuel consumption (economy) - extra urban (LPG) (NEDC)',
 'Fuel consumption (economy) - extra urban (LPG) (NEDC, WLTP equivalent)',
 'Fuel consumption (economy) - extra urban (CNG) (NEDC, WLTP equivalent)']]
#print(df_fuel_extra_urban)

### Combined

In [28]:
df_fuel_combined.head()

,Fuel consumption (economy) - combined,Fuel consumption (economy) - combined (NEDC),Fuel consumption (economy) - combined (LPG),"Fuel consumption (economy) - combined (NEDC, WLTP equivalent)",Fuel consumption (economy) - combined (CNG) (NEDC),Fuel consumption (economy) - combined (CNG),Fuel consumption (economy) - combined (Ethanol - E85),Fuel consumption (economy) - combined (LPG) (NEDC),"Fuel consumption (economy) - combined (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)"
0,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,6.5 l/100 km 36.19 US mpg43.46 UK mpg15.38 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#Functions to use in the cleaning part
def norm_fuel_grab(dfserie):
    return " ".join(dfserie.name.split()[3:]).replace('-','').rstrip().lstrip()
def fuel_clean(dfserie):
    return " ".join(dfserie.split()[0:1]).replace('combined','')

In [30]:
#Building the sum columns with all values adding the norm at the end of the string value
for column in df_fuel_combined.columns:
    df_fuel_combined[column] = df_fuel_combined[column].copy().apply(lambda item: fuel_clean(item) if (not isinstance(item, float)) else item)
    df_fuel_combined[column] = df_fuel_combined[[column]].copy().apply(lambda item: item +' ' +norm_fuel_grab(item)) 
df_fuel_combined['fuel_combined_clean'] = df_fuel_combined.fillna('').sum(axis=1).copy()
df_fuel_combined['fuel_combined_clean'] = df_fuel_combined['fuel_combined_clean'].apply(lambda item: np.nan if len(item) ==0 else item)
df_fuel_combined.head()

#df_co2[column] = df_gev[[column]].apply(lambda item: item + co2_grab(item))


C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/3587289883.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_combined[column] = df_fuel_combined[column].copy().apply(lambda item: fuel_clean(item) if (not isinstance(item, float)) else item)
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/3587289883.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_combined[column] = df_fuel_combined[[column]].copy().apply(lambda item: item +' ' +norm_fuel_grab(item))
C:\Users\UTILIZ~1\AppData\Lo

,Fuel consumption (economy) - combined,Fuel consumption (economy) - combined (NEDC),Fuel consumption (economy) - combined (LPG),"Fuel consumption (economy) - combined (NEDC, WLTP equivalent)",Fuel consumption (economy) - combined (CNG) (NEDC),Fuel consumption (economy) - combined (CNG),Fuel consumption (economy) - combined (Ethanol - E85),Fuel consumption (economy) - combined (LPG) (NEDC),"Fuel consumption (economy) - combined (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)",fuel_combined_clean
0,6.6 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.6 combined
1,6.4 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4 combined
2,6.6 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.6 combined
3,6.4 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4 combined
4,NaN,6.5 combined (NEDC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.5 combined (NEDC)


In [31]:
#Adding the new column with unit in the title name
df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean'].apply(lambda item: item.split()[0] if (not isinstance(item, float)) else item)
#and (item.split()[0].find('-') == -1 or item.split()[0].find('+') == -1 )
df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean (l/100km)'].apply(lambda item: item if (not isinstance(item, float) and item.replace('+','-').find('-') == -1) else item) ##. was float(item)   #and item.isdigit()) else item) #.str.isnumeric()
df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean (l/100km)'].apply(lambda item: basic_split_bin(item) if (not isinstance(item, float)) else item)
df_fuel_combined.head()

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/4144864261.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean'].apply(lambda item: item.split()[0] if (not isinstance(item, float)) else item)
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/4144864261.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean (l/100km)'].apply(l

,Fuel consumption (economy) - combined,Fuel consumption (economy) - combined (NEDC),Fuel consumption (economy) - combined (LPG),"Fuel consumption (economy) - combined (NEDC, WLTP equivalent)",Fuel consumption (economy) - combined (CNG) (NEDC),Fuel consumption (economy) - combined (CNG),Fuel consumption (economy) - combined (Ethanol - E85),Fuel consumption (economy) - combined (LPG) (NEDC),"Fuel consumption (economy) - combined (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)",fuel_combined_clean,fuel_combined_clean (l/100km)
0,6.6 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.6 combined,6.6
1,6.4 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4 combined,6.4
2,6.6 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.6 combined,6.6
3,6.4 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4 combined,6.4
4,NaN,6.5 combined (NEDC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.5 combined (NEDC),6.5


In [32]:
# df_fuel_combined[df_fuel_combined['fuel_combined_clean (l/100km)'].str.contains('5.5.') == True][100:150]
# df_fuel_combined[df_fuel_combined['fuel_combined_clean (l/100km)'] == '5.5.-5.4']
# df_fuel_combined.iloc[23012]

In [33]:
#for value in df_fuel_combined['fuel_combined_clean (l/100km)']:
 #   if type(value) == list:
  #      print(value)

In [34]:
#Code checking
#for value in df_fuel_combined['fuel_combined_clean (l/100km)']:
 #   if isinstance(value,str) and len(value)> 3:
 #       print(value)

In [35]:
df_fuel_combined.iloc[26701:]

,Fuel consumption (economy) - combined,Fuel consumption (economy) - combined (NEDC),Fuel consumption (economy) - combined (LPG),"Fuel consumption (economy) - combined (NEDC, WLTP equivalent)",Fuel consumption (economy) - combined (CNG) (NEDC),Fuel consumption (economy) - combined (CNG),Fuel consumption (economy) - combined (Ethanol - E85),Fuel consumption (economy) - combined (LPG) (NEDC),"Fuel consumption (economy) - combined (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - combined (CNG) (NEDC, WLTP equivalent)",fuel_combined_clean,fuel_combined_clean (l/100km)
26701,5.5.-5.4 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.5.-5.4 combined,"[5.5., 5.4]"
26702,4.1 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.1 combined,4.1
26703,5.5 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.5 combined,5.5
26704,4.4-4.3 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.4-4.3 combined,"[4.4, 4.3]"
26705,4.3-4.2 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.3-4.2 combined,"[4.3, 4.2]"
...,...,...,...,...,...,...,...,...,...,...,...,...
43107,13 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13 combined,13
43108,13 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13 combined,13
43109,13 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13 combined,13
43110,13 combined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13 combined,13


In [36]:
#Function 2 float df_fuel_combined
# def conv2float(item):
#     if type(item) == list:
#         value_list = []
#         print(item)
#         for value in item:
#             print(value)
#             value = value.replace('..','.')
#             value_list += value.rstrip('.')
#             print(value_list)
#             if value.count('.') == 3:
#                 value_list += [".".join(value.split('.')[0:2]),".".join(value.split('.')[2:5])]
#                 print(value_list)
#         return [float(i) for i in value_list if i != '.']
#     elif isinstance(item,str):   
#         item = item.replace('..','.')
#         item = item.rstrip('.')
#         if len(item)> 5 and item.count('.')==2:
#             print(item)
#             # item = [".".join([item.split('.')[0],item.split('.')[1][0]]),".".join([item.split('.')[1][1:],item.split('.')[2]])]
#             item = [".".join([item.split('.')[0],item.split('.')[1][0]]),".".join([item.split('.')[1][1:],item.split('.')[2]])]
#             print(item)
#             return [float(i) for i in item if i != '.']
#         return float(item)

In [37]:
#Floating every number to be able to do calculation
# df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean (l/100km)'].apply(lambda item: conv2float(item) if isinstance(item, str) else item)

In [38]:
# for value in df_fuel_combined['fuel_combined_clean (l/100km)']:
#     if type(value)==float:
#         print(float(value))

In [39]:
#####New function
def fixpoints(item):
    if type(item) == str:
        item = item.replace('..', '.')
        item = item.split()[0]
        # print(item)
        if len(item)> 5 and item.count('.')==2:
            # print(item)
            # item = ".".join([item.split('.')[0],item.split('.')[1][0]]) + ".".join([item.split('.')[1][1:],item.split('.')[2]]
            item = item.split('.')[0]+'.'+item.split('.')[1][0]+ "-"+item.split('.')[1][1:]+'.'+item.split('.')[2]
            # print(type(item))
            # return [i for i in item if i != '.']
            return item
    return item


df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean'].apply(fixpoints)\
.apply(lambda item: [re.findall('\d+\.*\d*',x) for x in item.split('-')] if not isinstance(item,float) else [])\
.apply(lambda item: [i[0] if len(i)>0 else '' for i in item] if len(item)>0 else [])

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/3454668405.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_combined['fuel_combined_clean (l/100km)'] = df_fuel_combined['fuel_combined_clean'].apply(fixpoints)\


### Urban

In [40]:
df_fuel_urban.head()

,Fuel consumption (economy) - urban,Fuel consumption (economy) - urban (NEDC),Fuel consumption (economy) - urban (LPG),"Fuel consumption (economy) - urban (NEDC, WLTP equivalent)",Fuel consumption (economy) - urban (CNG) (NEDC),Fuel consumption (economy) - urban (CNG),Fuel consumption (economy) - urban (Ethanol - E85),Fuel consumption (economy) - urban (LPG) (NEDC),"Fuel consumption (economy) - urban (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - urban (CNG) (NEDC, WLTP equivalent)"
0,9.1 l/100 km 25.85 US mpg31.04 UK mpg10.99 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.5 l/100 km 27.67 US mpg33.23 UK mpg11.76 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9.1 l/100 km 25.85 US mpg31.04 UK mpg10.99 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.5 l/100 km 27.67 US mpg33.23 UK mpg11.76 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,8.4 l/100 km 28 US mpg33.63 UK mpg11.9 km/l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
#Building the sum columns with all values adding the norm at the end of the string value
for column in df_fuel_urban.columns:
    df_fuel_urban[column] = df_fuel_urban[column].copy().apply(lambda item: fuel_clean(item) if (not isinstance(item, float)) else item)
    df_fuel_urban[column] = df_fuel_urban[[column]].copy().apply(lambda item: item +' ' +norm_fuel_grab(item)) 
df_fuel_urban['fuel_urban_clean'] = df_fuel_urban.fillna('').sum(axis=1).copy()
df_fuel_urban['fuel_urban_clean'] = df_fuel_urban['fuel_urban_clean'].apply(lambda item: np.nan if len(item) ==0 else item)


#Adding the new column with unit in the title name
df_fuel_urban['fuel_urban_clean (l/100km)'] = df_fuel_urban['fuel_urban_clean'].apply(lambda item: item.split()[0] if (not isinstance(item, float)) else item)
#and (item.split()[0].find('-') == -1 or item.split()[0].find('+') == -1 )
df_fuel_urban['fuel_urban_clean (l/100km)'] = df_fuel_urban['fuel_urban_clean (l/100km)'].apply(lambda item: item if (not isinstance(item, float) and item.replace('+','-').find('-') == -1) else item) ##. was float(item)   #and item.isdigit()) else item) #.str.isnumeric()
df_fuel_urban['fuel_urban_clean (l/100km)'] = df_fuel_urban['fuel_urban_clean (l/100km)'].apply(lambda item: basic_split_bin(item) if (not isinstance(item, float)) else item)


#Floating every number to be able to do calculation
df_fuel_urban['fuel_urban_clean (l/100km)'].apply(lambda item: fixpoints(item) if not isinstance(item, float) else '')

df_fuel_urban.head()

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/3859620653.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_urban[column] = df_fuel_urban[column].copy().apply(lambda item: fuel_clean(item) if (not isinstance(item, float)) else item)
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/3859620653.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_urban[column] = df_fuel_urban[[column]].copy().apply(lambda item: item +' ' +norm_fuel_grab(item))
C:\Users\UTILIZ~1\AppData\Local\Temp/ipy

,Fuel consumption (economy) - urban,Fuel consumption (economy) - urban (NEDC),Fuel consumption (economy) - urban (LPG),"Fuel consumption (economy) - urban (NEDC, WLTP equivalent)",Fuel consumption (economy) - urban (CNG) (NEDC),Fuel consumption (economy) - urban (CNG),Fuel consumption (economy) - urban (Ethanol - E85),Fuel consumption (economy) - urban (LPG) (NEDC),"Fuel consumption (economy) - urban (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - urban (CNG) (NEDC, WLTP equivalent)",fuel_urban_clean,fuel_urban_clean (l/100km)
0,9.1 urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.1 urban,9.1
1,8.5 urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.5 urban,8.5
2,9.1 urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.1 urban,9.1
3,8.5 urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.5 urban,8.5
4,NaN,8.4 urban (NEDC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.4 urban (NEDC),8.4


In [42]:
df_fuel_urban['fuel_urban_clean (l/100km)'] = df_fuel_urban['fuel_urban_clean (l/100km)'].apply(fixpoints)\
.apply(lambda item: [re.findall('\d+\.*\d*',x) for x in item.split('-')] if not isinstance(item,(float,list)) else [])\
.apply(lambda item: [i[0] if len(i)>0 else '' for i in item] if len(item)>0 else [])

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/2407303911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_urban['fuel_urban_clean (l/100km)'] = df_fuel_urban['fuel_urban_clean (l/100km)'].apply(fixpoints)\


### Extra_urban

In [43]:
#Building the sum columns with all values adding the norm at the end of the string value
for column in df_fuel_extra_urban.columns:
    df_fuel_extra_urban[column] = df_fuel_extra_urban[column].copy().apply(lambda item: fuel_clean(item) if (not isinstance(item, float)) else item)
    df_fuel_extra_urban[column] = df_fuel_extra_urban[[column]].copy().apply(lambda item: item +' ' +norm_fuel_grab(item)) 
df_fuel_extra_urban['fuel_extra_urban_clean'] = df_fuel_extra_urban.fillna('').sum(axis=1).copy()
df_fuel_extra_urban['fuel_extra_urban_clean'] = df_fuel_extra_urban['fuel_extra_urban_clean'].apply(lambda item: np.nan if len(item) ==0 else item)


#Adding the new column with unit in the title name
df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'] = df_fuel_extra_urban['fuel_extra_urban_clean'].apply(lambda item: item.split()[0] if (not isinstance(item, float)) else item)
#and (item.split()[0].find('-') == -1 or item.split()[0].find('+') == -1 )
df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'] = df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'].apply(lambda item: item if (not isinstance(item, float) and item.replace('+','-').find('-') == -1) else item) ##. was float(item)   #and item.isdigit()) else item) #.str.isnumeric()
df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'] = df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'].apply(lambda item: basic_split_bin(item) if (not isinstance(item, float)) else item)


#Floating every number to be able to do calculation
df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'].apply(lambda item: fixpoints(item) if not isinstance(item, float) else '')

df_fuel_extra_urban.head()


C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/52972107.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_extra_urban[column] = df_fuel_extra_urban[column].copy().apply(lambda item: fuel_clean(item) if (not isinstance(item, float)) else item)
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/52972107.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_extra_urban[column] = df_fuel_extra_urban[[column]].copy().apply(lambda item: item +' ' +norm_fuel_grab(item))
C:\Users\UTILIZ~1\Ap

,Fuel consumption (economy) - extra urban,Fuel consumption (economy) - extra urban (NEDC),Fuel consumption (economy) - extra urban (LPG),"Fuel consumption (economy) - extra urban (NEDC, WLTP equivalent)",Fuel consumption (economy) - extra urban (CNG) (NEDC),Fuel consumption (economy) - extra urban (CNG),Fuel consumption (economy) - extra urban (Ethanol - E85),Fuel consumption (economy) - extra urban (LPG) (NEDC),"Fuel consumption (economy) - extra urban (LPG) (NEDC, WLTP equivalent)","Fuel consumption (economy) - extra urban (CNG) (NEDC, WLTP equivalent)",fuel_extra_urban_clean,fuel_extra_urban_clean (l/100km)
0,5.2 extra urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2 extra urban,5.2
1,5.1 extra urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.1 extra urban,5.1
2,5.1 extra urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.1 extra urban,5.1
3,5.1 extra urban,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.1 extra urban,5.1
4,NaN,5.4 extra urban (NEDC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.4 extra urban (NEDC),5.4


In [44]:
df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'] = df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'].apply(fixpoints)\
.apply(lambda item: [re.findall('\d+\.*\d*',x) for x in item.split('-')] if not isinstance(item,(float, list)) else [])\
.apply(lambda item: [i[0] if len(i)>0 else '' for i in item] if len(item)>0 else [])

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/65901365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'] = df_fuel_extra_urban['fuel_extra_urban_clean (l/100km)'].apply(fixpoints)\


## Final dataframe

In [45]:
#First merge
df_final = df_gev_co2.merge(df_fuel_combined[['fuel_combined_clean','fuel_combined_clean (l/100km)']],left_index=True,right_index=True)
df_final = df_final.merge(df_fuel_urban[['fuel_urban_clean','fuel_urban_clean (l/100km)']],left_index=True,right_index=True)
df_final = df_final.merge(df_fuel_extra_urban[['fuel_extra_urban_clean','fuel_extra_urban_clean (l/100km)']],left_index=True,right_index=True)
df_final.head()

,Brand,Model,Generation,Modification (Engine),Body type,Fuel tank capacity,Fuel consumption (economy) - combined,Fuel Type,CO2 emissions,Power,co2_clean,co2_nb,co2_unit,Fuel_tank_capacity_clean_(liters),fuel_combined_clean,fuel_combined_clean (l/100km),fuel_urban_clean,fuel_urban_clean (l/100km),fuel_extra_urban_clean,fuel_extra_urban_clean (l/100km)
0,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp) Automatic,Roadster,45 l 11.89 US gal | 9.9 UK gal,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,Petrol (Gasoline),153 g/km,170 Hp @ 5500 rpm.,153 g/km,153.0,g/km,45.0,6.6 combined,[6.6],9.1 urban,[9.1],5.2 extra urban,[5.2]
1,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp),Roadster,45 l 11.89 US gal | 9.9 UK gal,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,Petrol (Gasoline),148 g/km,170 Hp @ 5500 rpm.,148 g/km,148.0,g/km,45.0,6.4 combined,[6.4],8.5 urban,[8.5],5.1 extra urban,[5.1]
2,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp) Automatic,Cabriolet,45 l 11.89 US gal | 9.9 UK gal,6.6 l/100 km 35.64 US mpg42.8 UK mpg15.15 km/l,Petrol (Gasoline),153 g/km,170 Hp @ 5500 rpm.,153 g/km,153.0,g/km,45.0,6.6 combined,[6.6],9.1 urban,[9.1],5.1 extra urban,[5.1]
3,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp),Cabriolet,45 l 11.89 US gal | 9.9 UK gal,6.4 l/100 km 36.75 US mpg44.14 UK mpg15.63 km/l,Petrol (Gasoline),148 g/km,170 Hp @ 5500 rpm.,148 g/km,148.0,g/km,45.0,6.4 combined,[6.4],8.5 urban,[8.5],5.1 extra urban,[5.1]
4,Abarth,500,500C,1.4 T-Jet (140 Hp) MTA,Cabriolet,35 l 9.25 US gal | 7.7 UK gal,NaN,Petrol (Gasoline),NaN,140 Hp @ 5500 rpm.,151 g/km(NEDC),151.0,g/km(NEDC),35.0,6.5 combined (NEDC),[6.5],8.4 urban (NEDC),[8.4],5.4 extra urban (NEDC),[5.4]


In [46]:
df_gev_beev = df_final[['Brand','Model','Generation','Modification (Engine)','Body type','Fuel Type','Power','co2_nb','co2_unit','Fuel_tank_capacity_clean_(liters)','fuel_combined_clean','fuel_combined_clean (l/100km)','fuel_urban_clean','fuel_urban_clean (l/100km)','fuel_extra_urban_clean', 'fuel_extra_urban_clean (l/100km)']]
df_gev_beev.head()

,Brand,Model,Generation,Modification (Engine),Body type,Fuel Type,Power,co2_nb,co2_unit,Fuel_tank_capacity_clean_(liters),fuel_combined_clean,fuel_combined_clean (l/100km),fuel_urban_clean,fuel_urban_clean (l/100km),fuel_extra_urban_clean,fuel_extra_urban_clean (l/100km)
0,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp) Automatic,Roadster,Petrol (Gasoline),170 Hp @ 5500 rpm.,153.0,g/km,45.0,6.6 combined,[6.6],9.1 urban,[9.1],5.2 extra urban,[5.2]
1,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp),Roadster,Petrol (Gasoline),170 Hp @ 5500 rpm.,148.0,g/km,45.0,6.4 combined,[6.4],8.5 urban,[8.5],5.1 extra urban,[5.1]
2,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp) Automatic,Cabriolet,Petrol (Gasoline),170 Hp @ 5500 rpm.,153.0,g/km,45.0,6.6 combined,[6.6],9.1 urban,[9.1],5.1 extra urban,[5.1]
3,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp),Cabriolet,Petrol (Gasoline),170 Hp @ 5500 rpm.,148.0,g/km,45.0,6.4 combined,[6.4],8.5 urban,[8.5],5.1 extra urban,[5.1]
4,Abarth,500,500C,1.4 T-Jet (140 Hp) MTA,Cabriolet,Petrol (Gasoline),140 Hp @ 5500 rpm.,151.0,g/km(NEDC),35.0,6.5 combined (NEDC),[6.5],8.4 urban (NEDC),[8.4],5.4 extra urban (NEDC),[5.4]


In [47]:
# df_gev_beev.fillna('N/A', inplace = True)

In [48]:
# for values in df_gev_beev['co2_unit']:
#     # if type(values) == str:
#      values = values.split(' ')
#      print(values)
#      co2_norm = re.findall('\w', values)
#      print(co2_norm)

In [49]:
# for values in df_gev_beev['fuel_combined_clean']:
    # if type(value) == str:
    #  norm = re.findall('\w{4,}', values)
    #  if len(norm) > 1:
    #   print(norm[1:])

In [50]:
def remove_list_fuel(value):
    if type(value) == list:
        if len(value) == 1 or '' in value:
            value = value[0]
            return float(value)
        elif len(value) == 2:
            return(sum([float(x)/2 for x in value]))
    else:
        return float(value)

def remove_list_tank(item):
    if type(item) == list:
        if len(item) == 2:
            return(sum([float(x)/2 for x in item]))
    else:
        return float(item)
            
            
df_gev_beev['fuel_combined_clean (l/100km)'] = df_gev_beev['fuel_combined_clean (l/100km)'].apply(lambda x : remove_list_fuel(x))
df_gev_beev['Fuel_tank_capacity_clean_(liters)'] = df_gev_beev['Fuel_tank_capacity_clean_(liters)'].apply(lambda x : remove_list_tank(x))
df_gev_beev['fuel_urban_clean (l/100km)'] = df_gev_beev['fuel_urban_clean (l/100km)'].apply(lambda x : remove_list_fuel(x))
df_gev_beev['fuel_extra_urban_clean (l/100km)'] = df_gev_beev['fuel_extra_urban_clean (l/100km)'].apply(lambda x : remove_list_fuel(x))

C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/121413124.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gev_beev['fuel_combined_clean (l/100km)'] = df_gev_beev['fuel_combined_clean (l/100km)'].apply(lambda x : remove_list_fuel(x))
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/121413124.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gev_beev['Fuel_tank_capacity_clean_(liters)'] = df_gev_beev['Fuel_tank_capacity_clean_(liters)'].apply(lambda x : remove_list_tank(x))
C:\Users\UTI

In [51]:
#####            RENAMING AND CALCULATIONS            ###########


df_gev_beev.rename(columns={'co2_nb' : 'CO2 emissions (g/km)', 'Fuel_tank_capacity_clean_(liters)': 'Fuel tank capacity (l)', 'fuel_combined_clean (l/100km)':'Fuel consumption - combined (l/100km)'
, 'fuel_urban_clean (l/100km)':'Fuel consumption - urban (l/100km)', 'fuel_extra_urban_clean (l/100km)': 'Fuel consumption - extra urban (l/100km)'}, inplace = True)
df_gev_beev['Power (hp)'] = df_gev_beev['Power'].apply(lambda x: str(x).split(' ')[0])
df_gev_beev['Range (km)'] = df_gev_beev[['Fuel tank capacity (l)','Fuel consumption - combined (l/100km)']].apply(lambda x : float(x[0])/float(x[1])*100, axis = 1)

C:\Users\Utilizador\anaconda3\envs\Safe space\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/1383351384.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gev_beev['Power (hp)'] = df_gev_beev['Power'].apply(lambda x: str(x).split(' ')[0])
C:\Users\UTILIZ~1\AppData\Local\Temp/ipykernel_18124/1383351384.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [52]:
df_gev_beev.head(10)

,Brand,Model,Generation,Modification (Engine),Body type,Fuel Type,Power,CO2 emissions (g/km),co2_unit,Fuel tank capacity (l),fuel_combined_clean,Fuel consumption - combined (l/100km),fuel_urban_clean,Fuel consumption - urban (l/100km),fuel_extra_urban_clean,Fuel consumption - extra urban (l/100km),Power (hp),Range (km)
0,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp) Automatic,Roadster,Petrol (Gasoline),170 Hp @ 5500 rpm.,153.0,g/km,45.0,6.6 combined,6.6,9.1 urban,9.1,5.2 extra urban,5.2,170,681.818182
1,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp),Roadster,Petrol (Gasoline),170 Hp @ 5500 rpm.,148.0,g/km,45.0,6.4 combined,6.4,8.5 urban,8.5,5.1 extra urban,5.1,170,703.125000
2,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp) Automatic,Cabriolet,Petrol (Gasoline),170 Hp @ 5500 rpm.,153.0,g/km,45.0,6.6 combined,6.6,9.1 urban,9.1,5.1 extra urban,5.1,170,681.818182
3,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp),Cabriolet,Petrol (Gasoline),170 Hp @ 5500 rpm.,148.0,g/km,45.0,6.4 combined,6.4,8.5 urban,8.5,5.1 extra urban,5.1,170,703.125000
4,Abarth,500,500C,1.4 T-Jet (140 Hp) MTA,Cabriolet,Petrol (Gasoline),140 Hp @ 5500 rpm.,151.0,g/km(NEDC),35.0,6.5 combined (NEDC),6.5,8.4 urban (NEDC),8.4,5.4 extra urban (NEDC),5.4,140,538.461538
5,Abarth,500,500C,1.4 T-Jet (135 Hp),Cabriolet,Petrol (Gasoline),135 Hp @ 5500 rpm.,155.0,g/km(NEDC),35.0,6.5 combined (NEDC),6.5,8.5 urban (NEDC),8.5,5.4 extra urban (NEDC),5.4,135,538.461538
6,Abarth,500,500,1.4 T-Jet (140 Hp) MTA,Hatchback,Petrol (Gasoline),140 Hp @ 5500 rpm.,151.0,g/km(NEDC),35.0,6.5 combined (NEDC),6.5,8.4 urban (NEDC),8.4,5.4 extra urban (NEDC),5.4,140,538.461538
7,Abarth,500,500,1.4 T-Jet (140 Hp) MTA,Hatchback,Petrol (Gasoline),140 Hp @ 5500 rpm.,150.0,g/km(NEDC),35.0,6.5 combined (NEDC),6.5,8.5 urban (NEDC),8.5,5.3 extra urban (NEDC),5.3,140,538.461538
8,Abarth,500,500,1.4 T-Jet (135 Hp),Hatchback,Petrol (Gasoline),135 Hp @ 5500 rpm.,155.0,g/km(NEDC),35.0,6.5 combined (NEDC),6.5,8.5 urban (NEDC),8.5,5.4 extra urban (NEDC),5.4,135,538.461538
9,Abarth,500,500,1.4 T-Jet (135 Hp),Hatchback,Petrol (Gasoline),135 Hp @ 5500 rpm.,155.0,g/km(NEDC),35.0,6.5 combined (NEDC),6.5,8.5 urban (NEDC),8.5,5.4 extra urban (NEDC),5.4,135,538.461538


In [53]:
df_gev_beev.to_csv("df_gev_beev.csv",index = False)

In [54]:
pd.read_csv("df_gev_beev.csv")

C:\Users\Utilizador\anaconda3\envs\Safe space\lib\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Brand,Model,Generation,Modification (Engine),Body type,Fuel Type,Power,CO2 emissions (g/km),co2_unit,Fuel tank capacity (l),fuel_combined_clean,Fuel consumption - combined (l/100km),fuel_urban_clean,Fuel consumption - urban (l/100km),fuel_extra_urban_clean,Fuel consumption - extra urban (l/100km),Power (hp),Range (km)
0,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp) Automatic,Roadster,Petrol (Gasoline),170 Hp @ 5500 rpm.,153.0,g/km,45.0,6.6 combined,6.6,9.1 urban,9.1,5.2 extra urban,5.2,170.0,681.818182
1,Abarth,124 Spider,124 GT,1.4 MultiAir (170 Hp),Roadster,Petrol (Gasoline),170 Hp @ 5500 rpm.,148.0,g/km,45.0,6.4 combined,6.4,8.5 urban,8.5,5.1 extra urban,5.1,170.0,703.125000
2,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp) Automatic,Cabriolet,Petrol (Gasoline),170 Hp @ 5500 rpm.,153.0,g/km,45.0,6.6 combined,6.6,9.1 urban,9.1,5.1 extra urban,5.1,170.0,681.818182
3,Abarth,124 Spider,124 Spider,1.4 MultiAir (170 Hp),Cabriolet,Petrol (Gasoline),170 Hp @ 5500 rpm.,148.0,g/km,45.0,6.4 combined,6.4,8.5 urban,8.5,5.1 extra urban,5.1,170.0,703.125000
4,Abarth,500,500C,1.4 T-Jet (140 Hp) MTA,Cabriolet,Petrol (Gasoline),140 Hp @ 5500 rpm.,151.0,g/km(NEDC),35.0,6.5 combined (NEDC),6.5,8.4 urban (NEDC),8.4,5.4 extra urban (NEDC),5.4,140.0,538.461538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43107,ZX,GrandTiger,GrandTiger,2.4 (126 Hp) 4x4,Pick-up,Petrol (Gasoline),126 Hp @ 5200 rpm.,NaN,NaN,65.0,13 combined,13.0,NaN,NaN,NaN,NaN,126.0,500.000000
43108,ZX,GrandTiger,GrandTiger,2.4 (126 Hp) 4x2,Pick-up,Petrol (Gasoline),126 Hp @ 5200 rpm.,NaN,NaN,65.0,13 combined,13.0,NaN,NaN,NaN,NaN,126.0,500.000000
43109,ZX,Landmark,Landmark,2.4 (126 Hp) 4x4,SUV,Petrol (Gasoline),126 Hp @ 5200 rpm.,NaN,NaN,73.0,13 combined,13.0,NaN,NaN,12 extra urban,12.0,126.0,561.538462
43110,ZX,Landmark,Landmark,2.4 (126 Hp) 4x2 Automatic,SUV,Petrol (Gasoline),126 Hp @ 5200 rpm.,NaN,NaN,73.0,13 combined,13.0,NaN,NaN,9 extra urban,9.0,126.0,561.538462


In [55]:
df_gev_beev['Fuel Type'].value_counts()

Petrol (Gasoline)               31074
Diesel                          11659
Petrol / CNG                      126
Petrol / Ethanol - E85            123
Petrol / LPG                      114
Mixture of two stroke engine       12
LPG                                 4
Name: Fuel Type, dtype: int64

In [56]:
df_gev_beev['Body type'].value_counts()

Sedan                                       10079
Hatchback                                    7993
Station wagon (estate)                       5834
Coupe                                        3885
SUV                                          3626
Minivan                                      2421
Pick-up                                      2259
Cabriolet                                    1752
Off-road vehicle                             1430
Minivan, MPV                                  649
MPV                                           647
SUV, Crossover                                446
Crossover                                     422
Roadster                                      364
Liftback                                      222
Fastback                                      189
Station wagon (estate), Crossover             175
Coupe - Cabriolet                             122
Targa                                          90
Grand Tourer                                   80


In [57]:
#Conversion of categories

# Sedan - Berline
# Compactes - Hacthback

In [58]:
#Mistakes examples
# df_fuel_combined.iloc[23012]
# df_fuel_combined.iloc[26701]